**Source Code of "Automatic Comic Style Transformation of Image"**


The program majorly runs and is tested through Google colab since it offers free GPU with High performance.

To use GPU, select "Runtime/Change runtime type" and choose GPU

All the "Test case" are removable

In [0]:
""" The code is adapted from
1.
Title: <Neural style transfer>
Author: <Mark Daoust, Billy Lamberta, Minho Heo, Yash Katariya >
Date: <2018>
Code version: <python3.6>
Type: <Python program>
Availability: <https://www.tensorflow.org/tutorials/generative/style_transfer>

2.
Title: <Neural Style Transfer: Creating Art with Deep Learning using tf.keras and eager execution>
Author: <Raymond Yuan>
Date: <2018-08-03>
Code version: <python3.6>
Type: <Blog>
Availability: <https://blog.tensorflow.org/2018/08/neural-style-transfer-creating-art-with-deep-learning.html>

"""

""" The code uses the following libraries:
1.
Title: <TensorFlow: Large-Scale Machine Learning on Heterogeneous Systems>
Author: <Martín Abadi, Ashish Agarwal, Paul Barham, Eugene Brevdo,
Zhifeng Chen, Craig Citro, Greg S. Corrado, Andy Davis,
Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Ian Goodfellow,
Andrew Harp, Geoffrey Irving, Michael Isard, Rafal Jozefowicz, Yangqing Jia,
Lukasz Kaiser, Manjunath Kudlur, Josh Levenberg, Dan Mané, Mike Schuster,
Rajat Monga, Sherry Moore, Derek Murray, Chris Olah, Jonathon Shlens,
Benoit Steiner, Ilya Sutskever, Kunal Talwar, Paul Tucker,
Vincent Vanhoucke, Vijay Vasudevan, Fernanda Viégas,
Oriol Vinyals, Pete Warden, Martin Wattenberg, Martin Wicke,
Yuan Yu, and Xiaoqiang Zheng>
Date: <2015>
Code version: <python3.6>
Type: <Software available from tensorflow.org>
Availability: <http://tensorflow.org/>

2.
Title: <Open Source Computer Vision Library>
Author: <OpenCV>
Date: <2015>
Type: <Software available from opencv.org>
Availability: <http://docs.opencv.org/>

3.
Title: <IPython: a system for interactive scientific computing>
Author: <Perez, Fernando and Granger, Brian E>
Date: <2007>
Journal: <Computing in Science & Engineering>
Volume: <9> 
Number: <3> 
Date: <2007>,
Publisher: <IEEE> 
Availability: <https://github.com/ipython/ipython>

"""

In [0]:
# Load the google drive before running the program 
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Please load the google drive before running the program
import os 

# Swtich the root directory to the image directory (Please adjust the path if required)
image_dir = "/content/drive/My Drive/source code/images"
os.chdir(image_dir)

# Check whether the directory loaded correctly. It should include style images and comic images
os.listdir(image_dir)

In [0]:
# Import necessary library
import tensorflow as tf
import matplotlib.pyplot as plt
import IPython.display
import numpy as np
import time
import cv2
from tensorflow.python.keras import *
from tensorflow.python.keras.preprocessing import image as kp_image
from PIL import Image

In [0]:
# Global values used in the model
# Path to styel and content images (Please adjust the path if required)
CONTENT_IMAGE='content_1.jpg' # You can change the content image through change this to filename of your own image
STYLE_IMAGE='comic_1.jpg' # You can change the style image through change this to filename of your own image
STYLE_IMAGE_2='comic_2.jpg'

# The size of image
HEIGHT=800
WEIGHT=600
CHANNEL=3

# The pre-trained model that we want to use for NST
MODEL=tf.keras.applications.vgg19# VGG is default in the paper
MODEL_VGG19=MODEL.VGG19(include_top=False,weights="imagenet")

# Content and Style layers that we want to extract the content and styler features from
CONTENT_LAYERS=['block3_conv3']
STYLE_LAYERS=['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']

# Weights for each layer and number of layers
STYLE_LAEYR_WEIGHTS=[0.1,0.1,0.35,0.35,0.1]
CONTENT_LAYER_WEIGHTS=[1]
NUM_CONTENT_LAYERS = len(CONTENT_LAYERS)
NUM_STYLE_LAYERS = len(STYLE_LAYERS)

# The average BGR of ImageNet since VGG-19 will substact each pixel by the average value for preprocessing
# These values are also used in deprocessing the image
NORM_MEANS=np.array([103.939, 116.779, 123.68])
MIN_VALS=-NORM_MEANS
MAX_VALS=255-NORM_MEANS

# Weights for each sections when computing the total loss
TOTAL_VAR_WEIGHT=1e3
STYLE_WEIGHT=1e1
CONTENT_WEIGHT=1e4
STYLE_WEIGHT_2=1e1

# Adust the proportion of noise when generate the original image
NOISE_RATIO=0

# Create kernel used for dilation/erosion
KERNEL=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))

# Used to store the best image for continuing the training
BEST_IMAGE_STORE=None

In [0]:
# Test case - output the model
vgg_test=MODEL_VGG19
vgg_test.summary()

In [0]:
def load_image(image):
  """load the image from the directory

  Args:
    image: the path of the image 

  Returns:
    image: image array with shape (1,Weight,Height,Channel)
  
  """
  image=Image.open(image)
  image=image.resize((WEIGHT, HEIGHT))
  image=kp_image.img_to_array(image)
  image=np.reshape(image,((1,)+np.shape(image)))
  return image

In [0]:
def load_and_process_image(image,enhance_edges=False):
  """load and pre-process the image from the directory

  For CNN model, it is beneficial to preprocess the input data for normalization.
  This function is used to realize loading and preprocessing the input data. If enhancem_edges
  is True, it will enhance the edges of the input image before preprocessing.
  
  Args:
    image: The path of the image
    enhance_edges: Control whether the edges of the input image should be enhanced
  
  Returns:
    image: A image arrary preprocessed based on the model

  """
  image=load_image(image)

  # Whether enhance the edges in the image
  if enhance_edges==True:
    image=image[0]
    enhanced_image,_,_=edge_enhance(image,image)
    plot_image_enhance=np.clip(enhanced_image,0,255).astype('uint8')
    image=np.reshape(plot_image_enhance,((1,)+np.shape(plot_image_enhance)))  

  image=MODEL.preprocess_input(image)
  
  return image

In [0]:
def deprocess_image(image):
  """Deprocess the image to obtain an image that is suitable for display

    Args:
      image: Pre-processed image

    Returns:
      image: Deprocessed image

  """ 
  # Remove the extra dimension added when we load the image
  if len(image.shape)==4:
    image=np.squeeze(image,0)
  
  # Perform the inverse of the preprocessiing step, these values are average values for BGR channels
  image[:,:,0]+=103.939
  image[:,:,1]+=116.779
  image[:,:,2]+=123.68
  image=image[:,:,::-1] # BGR<->RGB

  # Clip the generated imges' value to [0, 255] for display
  image=np.clip(image,0,255).astype('uint8')
  
  return image

In [0]:
# Test case - input image display
image_C=load_image(CONTENT_IMAGE)[0].astype('uint8')
image_S=load_image(STYLE_IMAGE)[0].astype('uint8')
image_S_2=load_image(STYLE_IMAGE_2)[0].astype('uint8')
plt.subplot(1,3,1)
plt.imshow(image_C)
plt.subplot(1,3,2)
plt.imshow(image_S)
plt.subplot(1,3,3)
plt.imshow(image_S_2)

In [0]:
def edge_extractor(image):
  """Extract edges from image

    Args:
      image: The path of image

    Returns:
      edges: The edges of the image

  """
  edges=cv2.GaussianBlur(image.astype(np.uint8),(3,3),4)
  edges=cv2.Canny(edges,200,300)

  # Invert the extracted edges map for further enhancement 
  edges=cv2.bitwise_not(edges)

  return edges

In [0]:
def edge_enhance(image_get_edge,image_result):
  """Enhance edges of image

  This function will enhance the edges of an image through adding
  the extracted edges back to the image after dilation.

    Args:
      image_get_edge: The path of image we want to extract the edges
      image_result: The image we want to enhance the edges.
    Returns:
      enhanced_image: The image after enhancement
      edges: The edge map after enhancement
      st_edges: The edge map before enhancement
  """
  edges=edge_extractor(image_get_edge)
  st_edges=edges

  # Since the edges map is inverted when extracted, erosion would be used for dilation purpose
  edges=cv2.morphologyEx(edges, cv2.MORPH_ERODE, KERNEL, iterations=1)

  # Cover the edges back to the image
  enhanced_image=cv2.bitwise_and(image_result,image_result,mask=edges)

  return enhanced_image,edges,st_edges

In [0]:
def get_model(model):
  """Create the model used to get the output values from intermediate layers

  The function will load the model and create a new model based on the layers
  that we choose to get the output values from

    Args:
      model: Yhe pre-trained model we want to use

    Returns:
      model: The new model created from the input model based on layers we choose

  """
  # Turn off the training since we want keep the weights the same.
  model.trainable=False

  # Get the output feature layers corresponding to the style and content layers we choose.
  style_outputs=[model.get_layer(name).output for name in STYLE_LAYERS]
  content_outputs=[model.get_layer(name).output for name in CONTENT_LAYERS]

  # Create the new model and turn off the traning.
  model=models.Model(model.input,style_outputs+content_outputs)
  for layer in model.layers:
    layer.trainable=False

  return model

In [0]:
# Test case - the layers read from the model
style_outputs=[vgg_test.get_layer(name).output for name in STYLE_LAYERS]
content_outputs=[vgg_test.get_layer(name).output for name in CONTENT_LAYERS]
style_outputs+content_outputs

In [0]:
# Test case - the model we created
#model=get_model(MODEL_VGG19)
#model.summary()

In [0]:
def get_content_feature(model,content_image,enhance_edges=False):
  """Compute the content features from the model.

  This function will load and preprocess the content image. 
  Then it will input it into the model to get the intermediate 
  outputs from the layer we choose.
  
  Arguments:
    model: The model that we use.
    content_image: The path to the content image.
    enhance_edges: Control whether the edges of the input image should be enhanced
    
  Returns:
    content_features: The content features. 
  """
  content_image=load_and_process_image(content_image,enhance_edges)
  content_outputs=model(content_image)

  # The reason of code style_layer[0] is to reduce the dimension, converting [1,n_h,n_w,n_c] into [n_h,n_w,n_c].
  # NUM_STYLE_LAYERS used to select the whether we want the style or content output since they are superimposed together when creating the model
  content_features=[content_layer[0] for content_layer in content_outputs[NUM_STYLE_LAYERS:]]
  
  return content_features

In [0]:
def get_style_feature(model,style_image):
  """Compute the style features from the model.

  This function will load and preprocess the style image. 
  Then it will input it into the model to get the intermediate 
  outputs from the layer we choose.
  
  Arguments:
    model: The model that we use.
    style_image: The path to the style image.
    
  Returns:
    content_features: The style features. 
  """
  style_image=load_and_process_image(style_image)
  style_outputs=model(style_image)
  style_features=[style_layer[0] for style_layer in style_outputs[:NUM_STYLE_LAYERS]]

  return style_features

In [0]:
def get_content_loss(base_content, target):
  """Calculate the content loss. 

    Args:
      base_content: The content output of generated image 
      target: The content output of content image
    
    Returns:
      loss: The content loss
  """
  loss=tf.reduce_mean(tf.square(base_content - target))

  return loss

In [0]:
def gram_matrix(image):
  """Calculate the Gram matrix  

    Args:
     image: the input tensor of the image.
    
    Returns:
      gram: The Gram matrix

  """ 
  # Flatten the image and set channel to the first.
  A=tf.reshape(image,shape=[-1,int(image.shape[-1])])

  n=tf.shape(A)[0]
  gram=tf.matmul(tf.transpose(A), A)
  gram=gram/tf.cast(n, tf.float32)
  
  return gram

In [0]:
def get_style_loss(base_style, gram_target):
  """Calculate the style loss. 

    Args:
      base_style: The style output of generated image 
      target: The style output of content image
    
    Returns:
      loss: The style loss
  """
  gram_style=gram_matrix(base_style)
  loss=tf.reduce_mean(tf.square(gram_style-gram_target))
  
  return loss

In [0]:
def get_loss_sum(model, style_weight,content_weight, generated_image, gram_style_features, 
                 content_features,style_layer_weights,content_layer_weights,gram_style_features_2=None,style_weight_2=0):
  """Calculate the loss sum

    Args:
      model: The CNN model used for extract the features
      style_weight: The proportion of style cost, also could serve as normalization
      content_weight: The proportion of content cost, also could serve as normalization
      generated_image: The generated image
      gram_style_features: The Gram matrix of style image
      content_features: The content features from content image
      style_layer_weights: Weights for each style layer
      content_layer_weights: Weights for each content layer
      gram_style_feature_2: The Gram matrix of style image 2, default to be None if there's no second style image
      style_weight_2: The proportion of style cost 2, also could serve as normalization,
              default to be 0 if there's no second style image
    
    Returns:
      loss: The sum of content cost, style cost and style cost 2 (if exists)
      style_cost: The sum of style costs (if style cost 2 exists)
      content_cost: The content cost
  """
  # Input the generated image to the model
  model_outputs=model(generated_image)
  
  # Separate the style features and content features generated through the model
  style_generated_outputs=model_outputs[:NUM_STYLE_LAYERS]
  content_generated_outputs=model_outputs[NUM_STYLE_LAYERS:]
  
  # The initial cost
  style_cost=0
  style_cost_2=0
  content_cost=0

  # Counters used for giving each ConvNet layer with corresponding weight
  style_layer_count=0
  style_layer_count_2=0
  content_layer_count=0
  
  # Calculate the sum of content loss from each layer
  dict_content=zip(content_features,content_generated_outputs)
  for target_content,generated_content in dict_content:
    content_cost+=content_layer_weights[content_layer_count]* get_content_loss(generated_content[0],target_content)
    content_layer_count+=1
    
  # Calculate the sum of style loss from each layer
  dict_style=zip(gram_style_features,style_generated_outputs) 
  for target_style,generated_style in dict_style:
    style_cost+=style_layer_weights[style_layer_count]*get_style_loss(generated_style[0],target_style)
    style_layer_count+=1

  # Calculate the sum of style loss 2 from each layer if there's second style image exists
  if gram_style_features_2 != None:
    dict_style_2=zip(gram_style_features_2,style_generated_outputs)
    for target_style_2,generated_style in dict_style_2:
      style_cost_2+=style_layer_weights[style_layer_count_2]*get_style_loss(generated_style[0],target_style_2)
      style_layer_count_2+=1
  
  # Calculate the total style loss
  style_cost=style_weight*style_cost+style_weight_2*style_cost_2
  content_cost=content_weight*content_cost

  # Calculate the total loss
  loss=style_cost+content_cost

  return loss,style_cost,content_cost

In [0]:
def generate_image(content_image,noise_ratio=NOISE_RATIO,enhance_edges=False):
  """Generate the initial image with or without white noise

  This function is used to generate the initial image. With high noise_ratio,
  The initial image would includes more white noise. If the noise_ratio is 0,
  The initial image would be the same as the content image.

    Args:
      content_image: The content image
      noise_ratio: The proportion of white noise in generated image
      enhance_edges: Control whether the edges of the input image should be enhanced
    
    Returns:
      generated_image: The generated image
  """
  image=load_and_process_image(content_image,enhance_edges)

  # Create a wthie noise image/content-image-realted image
  noise_image=np.random.uniform(-20,20,(1,HEIGHT,WEIGHT,CHANNEL))
  generated_image=noise_ratio*noise_image+(1-noise_ratio)*image

  return generated_image

In [0]:
def total_variation_loss(image,tv_weight):
  """Reduce the noise exists in the generated image

  This function will calculate the total variation loss in order to reduce
  the difference between neighboring pixels in order to reduce the noise and
  make the image look smooth.

    Args:
      image: The generated image
    
    Returns:
      loss: The total variation loss
  """
  x_var=image[:,:,1:,:]-image[:,:,:-1,:]
  y_var=image[:,1:,:,:]-image[:,:-1,:,:]
  loss=tv_weight*(tf.reduce_mean(x_var**2)+tf.reduce_mean(y_var**2))
  return loss

In [0]:
def one_training_step(cfg,opt,tv_weight):
  """Calculate and update the gradient

    Args:
      cfg: config including all required parameters for calculate the loss sum
      opt_learning_rate: learning rate
    
    Returns:
      loss: Total loss
      style_cost: Style loss
      content_cost: Content loss
      image: The generated after one update
      var_loss: Total variation loss
  """
  image=cfg['generated_image']

  # Apply gradient descent
  with tf.GradientTape() as tape: 
    all_loss=get_loss_sum(**cfg)
    var_loss=total_variation_loss(image,tv_weight)
    total_loss=all_loss[0]+var_loss
  grads=tape.gradient(total_loss,image)
  opt.apply_gradients([(grads,image)])

  clipped=tf.clip_by_value(image,MIN_VALS,MAX_VALS)
  image=image.assign(clipped)
  loss,style_cost,content_cost=all_loss

  return loss,style_cost,content_cost,image,var_loss

In [0]:
def convert_to_original_colors(content_image, generated_image):
  """Convert the generated image to its original color
  
  This function will adjust the color of generated image in YCbCr space.
  It would keep the luma component the same as generated image but convert
  its blue-difference and red-difference chroma components into original image's
  corresponding components.

    Args:
      content_image: The content image
      generated_image: The generated image
    
    Returns:
      converted_image: The generated image whose color has been converted to original color
  """
  content_image=load_image(content_image)[0].astype('uint8')
  
  # Switch the color sapce to YCbCr
  content_cvt=cv2.cvtColor(content_image,cv2.COLOR_BGR2YCR_CB)
  generated_cvt=cv2.cvtColor(generated_image,cv2.COLOR_BGR2YCR_CB)
  c1=cv2.split(generated_cvt)[0]
  c2=cv2.split(content_cvt)[1]
  c3=cv2.split(content_cvt)[2]
  merged=cv2.merge((c1,c2,c3))
  converted_image=cv2.cvtColor(merged,cv2.COLOR_YCR_CB2BGR).astype('uint8')

  return converted_image

In [0]:
def run_NST(model,content_image,style_image,style_layer_weights,content_layer_weights,
            num_iterations=1000,opt_learning_rate=1,content_weight=1,style_weight=1,
            continue_training=False,style_image_2=None,style_weight_2=0,tv_weight=0,enhance_edges=False): 
  """Implement the Neural Style Transfer

    Args:
      model: The model used for extracting the features
      content_image: The target content image
      style_image: The target style image
      style_layer_weights: The weights for each layers when calculating the style cost sum
      content_layer_weights: The weights for each layers when calculating the content cost sum
      num_iterations: Number of iteration
      opt_learning_rate: Learning rate
      content_weight: Weight of content cost
      style_weight: Weight of style cost
      continue_training: Whether to contiune the training based on the best gerenated image obtained
      style_image_2: The second target style image(if exists)
      style_weight_2: Weight of second style image
      tv_weight: Weight of total variation loss
      enhance_edges: Whether to enhance the edges of content image.
    
    Returns:
      best_image: The best generated image 
      best_loss: The lowest loss
      array_loss: The loss array during the training 
      array_style_loss: The style loss array during the training
      array_content_loss: The content loss array during the training
      array_TV_loss: The Total variation loss aray during the training
      images: Image set reflecting the process of training
      image_convert_color: The best generated image with color of target content image
  """
  global BEST_IMAGE_STORE

  # The second style image is empty in default  
  gram_style_features_2=None

  # Get the model used for feature extracting
  model=get_model(model) 

  # Get the style and content features from the chosen layer of model
  style_features=get_style_feature(model,style_image)
  content_features=get_content_feature(model,content_image,enhance_edges)

  # Calculate the Gram matrix for style feature
  gram_style_features=[gram_matrix(style_feature) for style_feature in style_features]

  # If second style image exists, get its feature and calculate its Gram matrix
  if style_image_2!=None:
    style_features_2=get_style_feature(model,STYLE_IMAGE_2)
    gram_style_features_2=[gram_matrix(style_feature_2) for style_feature_2 in style_features_2]
  
  # Set initial image for this training, if continue traning is True, it will use the current best generated result as the initial image.
  if continue_training==False:
    generated_image=generate_image(content_image,NOISE_RATIO,enhance_edges)
  elif BEST_IMAGE_STORE is None:
    generated_image=generate_image(content_image,NOISE_RATIO,enhance_edges)
  else:
    generated_image=BEST_IMAGE_STORE

  # Set generated_image as Variable for applying graident descent.
  generated_image=tf.Variable(generated_image,dtype=tf.float32)

  # Iteration counter.
  iter_count=1
  
  # Initialize the best loss and best image for storage in the future
  best_loss=float('inf')
  best_image=None
  
  # Create config including all required parameters for calculate the loss sum
  cfg={
      'model': model,
      'style_weight': style_weight,
      'content_weight':content_weight,
      'generated_image': generated_image,
      'gram_style_features': gram_style_features,
      'gram_style_features_2': gram_style_features_2,
      'content_features': content_features,
      'style_layer_weights':style_layer_weights,
      'content_layer_weights':content_layer_weights,
      'style_weight_2':style_weight_2
  }

  # Create optimizer
  opt=tf.optimizers.Adam(learning_rate=opt_learning_rate,beta_1=0.99,epsilon=1e-1)

  # Set the layout for displaying the generated image set at the end
  num_rows=2
  num_cols=5

  # Set the interval of recording the loss
  loss_store_interval=5

  # Set the interval of displaying generated image during the training
  display_interval=num_iterations/(num_rows*num_cols)

  # Initialize the Timmer
  start_time=time.time()
  end_time=time.time()
  global_start=time.time()
  global_end=time.time()   
  
  # Initialize the loss arrays and image set
  images=[]
  array_loss=[]
  array_style_loss=[]
  array_content_loss=[]
  array_TV_loss=[]

  # Start the training
  for i in range(num_iterations):
    # One step of training
    loss,style_cost,content_cost,generated_image,var_loss=one_training_step(cfg,opt,tv_weight)
    end_time=time.time() 
    
    # Store the best loss and best image
    if loss<best_loss:
      # Update best loss and best image from total loss. 
      best_loss=loss
      best_image=deprocess_image(generated_image.numpy())

    # Record the loss, skip the first 25 rounds since the loss is very huge at the begining, which will affect observation
    if i%loss_store_interval==0 and i>=25: 
      array_loss.append(loss)
      array_style_loss.append(style_cost)
      array_content_loss.append(content_cost)
      array_TV_loss.append(var_loss)

    # Display current training situation
    if i%display_interval==0:# Start timer
      average_time_interval=(end_time-start_time)/display_interval
      start_time=time.time()
      plot_image=deprocess_image(generated_image.numpy())

      # Store the image
      images.append(plot_image)

      # Display the generated image in training
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_image))

      # Display related data
      print('Iteration:{}'.format(i))        
      print('Total loss:{:.4e}'.format(loss))
      print('Style loss:{:.4e}'.format(style_cost))
      print('Content loss:{:.4e}'.format(content_cost))
      print('TV loss:{:.4e}'.format(var_loss))
      print('Average time for previous {:.1f} iterations:{:.4f}s'.format(display_interval,average_time_interval))

  # Record the final loss
  array_loss.append(loss)
  array_style_loss.append(style_cost)
  array_content_loss.append(content_cost)

  # Caculate total elapsed time
  global_end=time.time()

  # Display all images stored
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,image in enumerate(images):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(image)
      plt.xticks([])
      plt.yticks([])

  # Display the related data
  print('Total time:{:.4f}s'.format(global_end - global_start))
  print('Total loss:{:.4e}'.format(loss))
  print('Style loss:{:.4e}'.format(style_cost))
  print('Content loss:{:.4e}'.format(content_cost))
  print('TV loss:{:.4e}'.format(var_loss))
  print('Average time per iteation:{:.4f}s'.format((global_end - global_start)/num_iterations))
  
  # Convert the generated image to the color of target content image
  image_convert_color=convert_to_original_colors(content_image,best_image)

  # The following code is used for continuing the training during experiment, which is not necessary
  BEST_IMAGE_STORE=best_image
  BEST_IMAGE_STORE=np.reshape(best_image,((1,)+np.shape(best_image)))
  BEST_IMAGE_STORE=MODEL.preprocess_input(BEST_IMAGE_STORE)

  return best_image,best_loss,array_loss,array_style_loss,array_content_loss,array_TV_loss,images,image_convert_color

In [0]:
# Perform training， feel free to adjust the parameters
best_image,best_loss,array_loss,array_style_loss,array_content_loss,array_TV_loss,result_images,image_cvt=run_NST(
    model=MODEL_VGG19,
    content_image=CONTENT_IMAGE, 
    style_image=STYLE_IMAGE, 
    style_layer_weights=STYLE_LAEYR_WEIGHTS,
    content_layer_weights=CONTENT_LAYER_WEIGHTS,
    tv_weight=TOTAL_VAR_WEIGHT,
    num_iterations=1000,
    opt_learning_rate=5,
    content_weight=CONTENT_WEIGHT,
    style_weight=STYLE_WEIGHT,
    continue_training=False,
    enhance_edges=True,
    #style_image_2=STYLE_IMAGE_2, # Remove the Comment symbol to realize multiple style combination
    #style_weight_2=STYLE_WEIGHT_2 # Remove the Comment symbol to realize multiple style combination
    )

In [0]:
# Display the best image generated
IPython.display.display_png(Image.fromarray(best_image))

# Display the best image generated with color conversion
IPython.display.display_png(Image.fromarray(image_cvt))

In [0]:
def save_image_sets(images):
  """Save the image set

    Args:
      images: Image set including the generated images during the training
    
    Returns:
  """
  for i,image in enumerate(images):
    Image.fromarray(image).save('generated_image_'+str(i+1) +'.jpg')

In [0]:
def save_best_image(best_image):
  """Save the best image

    Args:
      best_image: Best image generated
    
    Returns:
  """
  Image.fromarray(best_image).save('generated_image.jpg')

In [0]:
def save_loss_plot(array_loss,array_style_loss,array_content_loss):
  """Plot and save the loss figures

    Args:
      array_loss: Array of total loss
      array_style_loss: Array of style loss
      array_content_loss: Array of content loss
    
    Returns:
  """
  plt.figure(figsize=(10,16))
  plt.subplot(3,1,1)
  plt.plot(array_loss)
  plt.title('total loss')
  plt.subplot(3,1,2)
  plt.plot(array_style_loss)
  plt.title('style loss')
  plt.subplot(3,1,3)
  plt.plot(array_content_loss)
  plt.title('content loss')
  plt.savefig('loss figure')
  plt.close


In [0]:
# Save results to the file
save_image_sets(result_images)
save_best_image(best_image)
save_loss_plot(array_loss,array_style_loss,array_content_loss)
save_best_image(image_cvt)

In [0]:
# Plot the total variation loss figures
plt.plot(array_TV_loss)
plt.title('Total variation loss')